In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json

import pandas as pd
import numpy as np

In [ ]:
urlBaseV1 = 'https://statsapi.mlb.com/api/v1/'
urlBaseV1_1 = 'https://statsapi.mlb.com/api/v1.1/'

Validar que es deporte 23 (ligas independeinte)

In [ ]:
data = requests.get(urlBaseV1 + 'sports/23').content
data = json.loads(data)
with open('deportes.json', 'w') as f:
    json.dump(data, f, indent=4)

Validar que la liga sea 125

In [ ]:
data = requests.get(urlBaseV1 + 'league/125').content
data = json.loads(data)
with open('liga.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los datos de las temporadas para la LMB

In [ ]:
data = requests.get(urlBaseV1 + 'seasons/all?leagueId=125').content
data = json.loads(data)
with open('temporadas.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los tipos de juegos

In [ ]:
data = requests.get(urlBaseV1 + 'gameTypes').content
data = json.loads(data)
with open('tiposJuegos.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los tipo at bat

In [ ]:
data = requests.get(urlBaseV1 + 'eventTypes').content
data = json.loads(data)
with open('tiposAtBat.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los tipos de lanzamientos

In [ ]:
data = requests.get(urlBaseV1 + 'pitchCodes').content
data = json.loads(data)
with open('tiposLanzamiento.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los tipos de status para los juegos

In [ ]:
data = requests.get(urlBaseV1 + 'gameStatus').content
data = json.loads(data)
with open('tiposStatusJuego.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener las diferentes posiciones

In [ ]:
data = requests.get(urlBaseV1 + 'positions').content
data = json.loads(data)
with open('posiciones.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener datos de los equipos actuales de la liga

In [ ]:
data = requests.get(urlBaseV1 + 'teams?leagueId=125').content
data = json.loads(data)
with open('equipos.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener la informacion de los diablos

In [ ]:
data = requests.get(urlBaseV1 + 'teams?teamId=532').content
data = json.loads(data)
with open('InformacionDiablos.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener roster diablos 2025

In [ ]:
data = requests.get(urlBaseV1 + 'teams/532/roster?season=2025').content
data = json.loads(data)
with open('rosterDiablos2025.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener informacion de gamboa (jugador diablos)

In [ ]:
data = requests.get(urlBaseV1 + 'people/585251/').content
data = json.loads(data)
with open('informacionGamboa.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener los partidos de una temporada (datos apartir de la temporada 2021)

In [ ]:
temporada = 2023
data = requests.get(urlBaseV1 + f'schedule?sportId=23&leageId=125&season={temporada}').content
data = json.loads(data)
with open(f'calendario{temporada}.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener informacion del primer juego de diablos 2025

In [ ]:
data = requests.get(urlBaseV1_1 + 'game/809656/feed/live').content
data = json.loads(data)
with open('partidoInaguralDiablos.json', 'w') as f:
    json.dump(data, f, indent=4)

Obtener informacion del primer at bat del juego

In [ ]:
df = pd.DataFrame(columns=['atBat', 'nLanzamiento', 'resultado', 'x', 'y'])
atBat = 1
for aB in range(atBat):
    dfAtBat = pd.DataFrame(columns=['atBat', 'nLanzamiento', 'resultado', 'x', 'y'])
    datosAtBat = data['liveData']['plays']['allPlays'][aB]
    lanzamientos = len(datosAtBat['playEvents'])
    nombreBateador = datosAtBat['matchup']['batter']['fullName']
    nombreLanzador = datosAtBat['matchup']['pitcher']['fullName']
    zonaTop = (datosAtBat['playEvents'][-1]['pitchData']['strikeZoneTop'])
    zonaBottom = (datosAtBat['playEvents'][-1]['pitchData']['strikeZoneBottom'])
    altura = zonaTop - zonaBottom
    contador = 0
    for lanzamiento in range(lanzamientos):
        # validar si el lanzamiento tiene el parametro 'call'
        if 'call' not in datosAtBat['playEvents'][lanzamiento]['details']:
            continue
        contador += 1
        x = -0.021 * (datosAtBat['playEvents'][lanzamiento]['pitchData']['coordinates']['x']) + 2.298
        y = -0.021 * (datosAtBat['playEvents'][lanzamiento]['pitchData']['coordinates']['y']) + 5.803
        dfAtBat.loc[len(dfAtBat)] = {'atBat': aB,
                           'nLanzamiento': contador,
                           'resultado': datosAtBat['playEvents'][lanzamiento]['details']['call']['description'],
                           'x': x,
                           'y': y}
        df.loc[len(df)] = {'atBat': aB,
                           'nLanzamiento': contador,
                           'resultado': datosAtBat['playEvents'][lanzamiento]['details']['call']['description'],
                           'x': x,
                           'y': y}

    plt.figure(figsize=(6, 6))
    plt.title(f'Pitcheos de {nombreLanzador} vs {nombreBateador} - Turno {aB + 1}')
    plt.scatter(dfAtBat['x'], dfAtBat['y'], c='blue', label='Pitcheos')

    # Agrega el número de pitcheo junto a cada punto
    for i, row in dfAtBat.iterrows():
        plt.text(row['x'], row['y'], str(row['nLanzamiento']), fontsize=9, ha='left', va='bottom')

    # Coordenadas de la zona de strike
    x_cuadro = [-0.7, -0.7, 0.7, 0.7, -0.7]
    y_cuadro = [zonaBottom, zonaTop, zonaTop, zonaBottom, zonaBottom]
    plt.plot(x_cuadro, y_cuadro, color='red', linestyle='-', label='Cuadro zona')

    # Lineas de la zona de strike
    tercioVertical = (zonaTop - zonaBottom) / 3
    tercioHorizontal = 1.4 / 3
    plt.plot([-0.7, 0.7], [zonaBottom + tercioVertical, zonaBottom + tercioVertical], color='red', linestyle='-')
    plt.plot([-0.7, 0.7], [zonaTop - tercioVertical, zonaTop - tercioVertical], color='red', linestyle='-')
    plt.plot([-0.7 + tercioHorizontal, -0.7 + tercioHorizontal], [zonaBottom, zonaTop], color='red', linestyle='-')
    plt.plot([0.7 - tercioHorizontal, 0.7 - tercioHorizontal], [zonaBottom, zonaTop], color='red', linestyle='-')

    # Etiquetas y limites
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 5)

    plt.show()
        